In [1]:
import sys
print(sys.version)
print(spark.version)

3.8.15 | packaged by conda-forge | (default, Nov 22 2022, 08:46:39) 
[GCC 10.4.0]
3.1.3


In [2]:
import os
import shutil
import io
import re
import warnings
from datetime import datetime
import pytz
import string
import pandas as pd
import numpy as np
pd.set_option('display.max_colwidth', None)
pd.reset_option('display.max_rows')
from itertools import compress 
from pyspark.sql.functions import *
from pyspark.sql import functions as f
from pyspark.sql.types import *
import seaborn as sns
import matplotlib.pyplot as plt
warnings.filterwarnings(action='ignore')

In [3]:
##Add "eagerEval.enabled" to beautify the way Spark DF is displayed
spark.conf.set("spark.sql.repl.eagerEval.enabled",True)
## To use legacy casting notation for date
spark.conf.set("spark.sql.legacy.timeParserPolicy","LEGACY")

In [4]:
from google.cloud import storage

In [5]:
def list_blobs(bucket_name, folder_name):
    gcs_client = storage.Client()
    bucket = gcs_client.bucket(bucket_name)
    blobs = list(bucket.list_blobs(prefix=folder_name))

    for blob in blobs:
        print(blob.name + '\t' + str(blob.size))

In [6]:
def list_blobs_pd(bucket_name, folder_name):
    gcs_client = storage.Client()
    bucket = gcs_client.bucket(bucket_name)
    blobs = list(bucket.list_blobs(prefix=folder_name))

    blob_name = []
    blob_size = []
    
    for blob in blobs:
        blob_name.append(blob.name)
        blob_size.append(blob.size)

    blobs_df = pd.DataFrame(list(zip(blob_name, blob_size)), columns=['Name','Size'])

    blobs_df.style.format({"Size": "{:,.0f}"}) 
    
    return blobs_df

In [7]:
def delete_folder(bucket_name, folder_name):
    gcs_client = storage.Client()
    bucket = gcs_client.bucket(bucket_name)
    blobs = list(bucket.list_blobs(prefix=folder_name))

    for blob in blobs:
        blob.delete()

### Read Data

In [8]:
tweets_filtered = spark.read.parquet('gs://msca-bdp-students-bucket/shared_data/pranavr569/tweets_filtered')

23/03/05 06:49:29 WARN org.apache.spark.sql.catalyst.util.package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


In [9]:
tweets_filtered.count()

13128895

### Performing Exploratory Data Analysis

In [11]:
tweets_filtered.describe()

summary,created_at,favorite_count,filter_level,id,id_str,in_reply_to_screen_name,in_reply_to_status_id,in_reply_to_status_id_str,in_reply_to_user_id,in_reply_to_user_id_str,lang,quote_count,quoted_status_id,quoted_status_id_str,quoted_text,reply_count,retweet_count,retweeted,retweeted_from,source,text,timestamp_ms,tweet_text,stripped,edu_count
count,13128895,13128895,13128895,13128895,13128895,1848930,1773774,1773774,1848930,1848930,13128895,13128895,852195,852195,850399,13128895,13128895,13128895,10009266,13128895,13128895,13128895,13128895,13128895,13128895
mean,null,0.0,null,1.565636553461320...,1.565636553461320...,Infinity,1.564648642702757...,1.564648642702757...,6.370169251329724...,6.370169251329724...,null,0.0,1.559759477994343...,1.559759477994343...,407.022,0.0,0.0,null,1.141119657774527...,null,null,1.662111817122399...,null,null,2.28552136337445
stddev,null,0.0,null,3.186911390426742...,3.186911390426742...,NaN,3.970924160205816...,3.970924160205816...,6.525627680746927...,6.525627680746927...,null,0.0,6.387547417489908...,6.387547417489908...,892.7624109582572,0.0,0.0,null,2.847440352376575E11,null,null,7.598188854270931E9,null,null,0.6493054825310967
min,Fri Apr 08 00:00:...,0,low,1511197336059342854,1511197336059342854,0,1248177489,1000413472356696065,12,1000002812087099394,en,0,22,1000290034715627520,! @NYCMayor⁩⁩’s j...,0,0,,,"<a href=""HTTP://b...",! Indoctrination ...,1649132496262,"! no mage today,...",😂😂😂😂😂😂?...,2
max,Wed Sep 28 23:59:...,0,low,1623308317287149569,1623308317287149569,zzzzzzz43088334,1623307653853192192,999350959057854464,1623253944557355008,999999616279089152,en,0,1623294703243341827,999670241025081344,󠇰 󠇰 󠇰 󠇰 It s...,0,0,RT,🅰️ Chemistry Ass...,"<a href=""https://...",🫶🏾🎓 @celebrati...,1675861837108,🫶🏾🎓 @celebrati...,🫶🏾🎓 @celebrati...,25


In [13]:
tweets_filtered.select([count(when(isnull(c), c)).alias(c) for c in tweets_filtered.columns]).toPandas().transpose()

,0
coordinates,13117111
created_at,0
display_text_range,10882583
entities,0
extended_entities,12698833
extended_tweet,10045021
favorite_count,0
favorited,0
filter_level,0
geo,13117111


In [14]:
# For non-struct columns
non_struct_cols = [col for col, t in tweets_filtered.dtypes if not t.startswith('struct')]
# For struct columns
struct_cols = [col for col, t in tweets_filtered.dtypes if t.startswith('struct')]

In [15]:
tweets_filtered.select(non_struct_cols).printSchema()

root
 |-- created_at: string (nullable = true)
 |-- display_text_range: array (nullable = true)
 |    |-- element: long (containsNull = true)
 |-- favorite_count: long (nullable = true)
 |-- favorited: boolean (nullable = true)
 |-- filter_level: string (nullable = true)
 |-- id: long (nullable = true)
 |-- id_str: string (nullable = true)
 |-- in_reply_to_screen_name: string (nullable = true)
 |-- in_reply_to_status_id: long (nullable = true)
 |-- in_reply_to_status_id_str: string (nullable = true)
 |-- in_reply_to_user_id: long (nullable = true)
 |-- in_reply_to_user_id_str: string (nullable = true)
 |-- is_quote_status: boolean (nullable = true)
 |-- lang: string (nullable = true)
 |-- possibly_sensitive: boolean (nullable = true)
 |-- quote_count: long (nullable = true)
 |-- quoted_status_id: long (nullable = true)
 |-- quoted_status_id_str: string (nullable = true)
 |-- quoted_text: string (nullable = true)
 |-- reply_count: long (nullable = true)
 |-- retweet_count: long (nullabl

In [16]:
tweets_filtered.select(['tweet_text', 'text', 'quoted_text', 'retweeted']).sample(0.1).limit(5)

tweet_text,text,quoted_text,retweeted
i tell college at...,RT @AdamBreneman8...,null,RT
children of polit...,RT @Naija_PR: Chi...,null,RT
new research from...,RT @UArkPsych: Ne...,null,RT
trek venture 2.0 ...,RT @MillionSmiles...,null,RT
@gamedaygarage i’...,RT @ChastityFallo...,null,RT


In [17]:
tweets_filtered.select(['id', 'id_str']).sample(0.1).limit(5)

id,id_str
1559580843131420672,1559580843131420672
1559580923452354560,1559580923452354560
1559580949389754368,1559580949389754368
1559581094650994689,1559581094650994689
1559581207846944769,1559581207846944769


In [18]:
tweets_filtered.select(['retweeted_from', 'retweeted']).sample(0.1).limit(5)

retweeted_from,retweeted
Melissa__x0xo,RT
adejimiadeniji,RT
null,
Luveniwai,RT
SonexStella,RT


In [19]:
tweets_filtered.select('extended_tweet').sample(0.1).limit(5)

extended_tweet
null
null
null
null
null


In [20]:
tweets_filtered.filter(f.col('retweeted_status').isNotNull()).count()

9172744

In [21]:
# Retweet_count only has zeros
tweets_filtered.select('retweet_count').distinct().limit(10)

retweet_count
0


In [22]:
# Retweet_count within retweeted_status has real numbers 
tweets_filtered.select('retweeted_status.retweet_count').distinct().limit(10)

retweet_count
6
2668
68
342
3384
9702
4456
6837
713
27596


In [23]:
tweets_filtered.select(['retweeted_from', 'retweeted']).limit(5)

retweeted_from,retweeted
AdamBreneman81,RT
thecreativeschl,RT
Naija_PR,RT
AfricaFactsZone,RT
null,


In [24]:
tweets_filtered.select('coordinates').printSchema()

root
 |-- coordinates: struct (nullable = true)
 |    |-- coordinates: array (nullable = true)
 |    |    |-- element: double (containsNull = true)
 |    |-- type: string (nullable = true)



In [25]:
tweets_filtered.filter(f.col('place.country').isNotNull()).count()

90449

In [26]:
tweets_filtered.filter(f.col('user.location').isNotNull()).count()

8135228

In [27]:
tweets_filtered.filter(f.col('user.location').isNotNull()).select('user.location').limit(10)

location
TorontoMetU - Tor...
Lagos Nigeria
"Lagos, Nigeria"
India
"Gurdaspur, Punjab"
he him 18
"Fayetteville, AR"
"Ayrshire, Scotland"
"pittsburgh, pa"
"Baltimore, MD"


### Filtering out the useful columns

In [9]:
tweet_cols = ["created_at", "retweeted_status", 'retweeted', 'in_reply_to_status_id_str',"id_str",
              "retweeted_from", "tweet_text","quoted_status"]

coordinate_cols = ["coordinates"]

user_cols = ["description","favourites_count","followers_count","friends_count","id_str",
            "name","protected","screen_name","statuses_count","verified","withheld_in_countries","location"]

ent_cols = ["hashtags"]

retweet_cols = ["retweet_count","user.name","user.description"]

quoted_cols = ["quote_count"]

tweets_df = tweets_filtered.select([*[col('user.' + col_name).alias('user_' + col_name) for col_name in user_cols],
                                  *[col(col_name).alias(col_name) for col_name in tweet_cols],
                                  *[col('entities.' + col_name).alias(col_name) for col_name in ent_cols],
                         *[col('coordinates.' + col_name).alias("tweet_coordinates") for col_name in coordinate_cols],
                            *[col('retweeted_status.' + col_name).alias(col_name) for col_name in retweet_cols]])\
.withColumn('created_at',to_timestamp(col('created_at'),'EEE MMM dd HH:mm:ss zzzzz yyyy'))

tweets_df.limit(5)

user_description,user_favourites_count,user_followers_count,user_friends_count,user_id_str,user_name,user_protected,user_screen_name,user_statuses_count,user_verified,user_withheld_in_countries,user_location,created_at,retweeted_status,retweeted,in_reply_to_status_id_str,id_str,retweeted_from,tweet_text,quoted_status,hashtags,tweet_coordinates,retweet_count,user.name,user.description
PSU | Pittsburg...,1170,1131,681,258994988,Gregg Garrity,false,GreggGarrity19,4157,false,[],null,2022-08-02 15:20:19,"{null, Tue Aug 02...",RT,null,1554487264255803392,AdamBreneman81,i tell college at...,null,[],null,13,Adam Breneman,"Host, @BrenemanSh..."
Toronto Metropoli...,3149,625,225,1920532658,StrongerTMU,false,StrongerTMU,5201,false,[],TorontoMetU - Tor...,2022-08-02 15:20:20,"{null, Wed Jul 27...",RT,null,1554487268446015490,thecreativeschl,in a conversation...,null,[],null,3,The Creative School,A dynamic faculty...
"Crypto, Music Blo...",5919,139,528,1363991889184227329,Henry,false,henryyyyyyyy_,1119,false,[],Lagos Nigeria,2022-08-02 15:20:28,"{null, Tue Aug 02...",RT,null,1554487301157408768,Naija_PR,children of polit...,null,[],null,218,Naija,Get to know the t...
just being me; Pr...,8373,1933,1956,3121506809,Skillful,false,skillful_makuo,10158,false,[],"Lagos, Nigeria",2022-08-02 15:20:30,"{null, Tue Aug 02...",RT,null,1554487309617299458,AfricaFactsZone,rwanda has increa...,null,[],null,1531,Africa Facts Zone,Africa's biggest ...
null,8099,118,268,2409456132,Gourav Dwivedi,false,gouravd17,3820,false,[],India,2022-08-02 15:20:32,null,,1554485667630960640,1554487319990013957,null,@dpradhanbjp sir ...,null,[],null,null,null,null


In [10]:
tweets_df = tweets_df.withColumnRenamed('coordinates', 'tweet_coordinates')\
.withColumnRenamed('user_id_str','user_id')\
.withColumnRenamed('user.name','retweeted_from_name')\
.withColumnRenamed('user.description','retweeted_from_description')\
.withColumnRenamed('id_str','tweet_id')

In [30]:
tweets_df.columns

['user_description',
 'user_favourites_count',
 'user_followers_count',
 'user_friends_count',
 'user_id',
 'user_name',
 'user_protected',
 'user_screen_name',
 'user_statuses_count',
 'user_verified',
 'user_withheld_in_countries',
 'user_location',
 'created_at',
 'retweeted_status',
 'retweeted',
 'in_reply_to_status_id_str',
 'tweet_id',
 'retweeted_from',
 'tweet_text',
 'quoted_status',
 'hashtags',
 'tweet_coordinates',
 'retweet_count',
 'retweeted_from_name',
 'retweeted_from_description']

### Boolean columns for Reply and Retweet Filtering

In [11]:
tweets_df = tweets_df.withColumn('in_reply_to_status_id_str', f.when(tweets_df['in_reply_to_status_id_str'].isNotNull(), True).otherwise(False))

In [32]:
tweets_df.select('in_reply_to_status_id_str').limit(10)

in_reply_to_status_id_str
false
true
false
false
false
false
false
true
true
false


In [12]:
# Change column name
tweets_df = tweets_df.withColumnRenamed('in_reply_to_status_id_str', 'is_reply')

In [34]:
tweets_df.filter(tweets_df['is_reply']=='true').count()

1773774

In [13]:
tweets_df = tweets_df.withColumn('retweeted', f.when(tweets_df['retweet_count'].isNotNull(), True).otherwise(False))

In [14]:
# Change column name
tweets_df = tweets_df.withColumnRenamed('retweeted', 'is_retweet')

In [37]:
# Check retweet_description
tweets_df.select(['is_retweet', 'retweeted_from_description', 'user_description']).limit(10)

is_retweet,retweeted_from_description,user_description
false,null,UI/UX Design stud...
true,logcast:imzarvinn...,I'm counting on y...
true,logcast:imzarvinn...,she/her ...
true,Speaking of Jesus...,"Physics PhD, soft..."
false,null,more than 1k+ wor...
true,Co-Founder/Lawyer...,null
false,null,more than 1k+ wor...
true,4th grade teacher...,✨3rd Grade teache...
false,null,Aristocrat
true,"Editor in Chief, ...",Molecular microbi...


In [ ]:
#Only when is_retweet is True should retweeted_from_description be non-null. In addition, retweeted_from_description should be different from user_description when it is non-null

In [38]:
# Check return zero rows
tweets_df.select(['is_retweet', 'retweeted_from_description']).filter(~(f.col('is_retweet')) & (f.col('retweeted_from_description').isNotNull())).show(10)

+----------+--------------------------+
|is_retweet|retweeted_from_description|
+----------+--------------------------+
+----------+--------------------------+



In [39]:
tweets_df.select('created_at').limit(10)

created_at
2022-08-23 22:55:01
2022-08-23 22:55:02
2022-08-23 22:55:03
2022-08-23 22:55:03
2022-08-23 22:55:03
2022-08-23 22:55:04
2022-08-23 22:55:09
2022-08-23 22:55:12
2022-08-23 22:55:12
2022-08-23 22:55:13


In [15]:
# Rename date columns
tweets_df = tweets_df.withColumnRenamed('created_at', 'tweet_date')

In [41]:
missing_ratios = {key:tweets_df.filter(f.col(key).isNull()).count() / 13128895 for key in tweets_df.columns}

In [42]:
missing_df = pd.DataFrame(
    {'cols': missing_ratios.keys(),
     'ratios': missing_ratios.values()}
)
missing_df

,cols,ratios
0,user_description,0.175609
1,user_favourites_count,0.000000
2,user_followers_count,0.000000
3,user_friends_count,0.000000
4,user_id,0.000000
5,user_name,0.000000
6,user_protected,0.000000
7,user_screen_name,0.000000
8,user_statuses_count,0.000000
9,user_verified,0.000000


In [43]:
unique_users = tweets_df.select('user_id').distinct().count()
unique_users

5242316

In [44]:
unique_tweets = tweets_df.select('tweet_id').distinct().count()
unique_tweets

13127771

In [46]:
tweets_df.select(['is_retweet', 'retweeted_from', 'retweet_count']).limit(10)

is_retweet,retweeted_from,retweet_count
true,coffee_puccino,4507
true,RepValDemings,143
false,null,null
true,chriseveleens,3
false,null,null
true,ochoshon,5
true,ifyouseekkae,2179
true,passionspen,252
false,null,null
true,alainnasimone,41


In [47]:
tweets_df.filter((f.col('retweeted_from').isNotNull()) & (f.col('is_retweet').isNull())).select('is_retweet', 'retweeted_from').limit(10)

is_retweet,retweeted_from


In [48]:
tweets_df.filter((f.col('retweeted_from').isNull()) & (f.col('is_retweet').isNotNull())).select('is_retweet', 'retweeted_from').limit(10)

is_retweet,retweeted_from
false,null
false,null
false,null
false,null
false,null
false,null
false,null
false,null
false,null
false,null


In [49]:
tweets_df.select(['user_statuses_count']).limit(10)

user_statuses_count
9168
1465
11535
38382
12376
556
1751
1793
2839
84429


In [50]:
#There is a relationship between is_retweet and retweeted_status 
tweets_df.select(['is_retweet', 'retweeted_status']).limit(10)

is_retweet,retweeted_status
true,"{null, Mon Aug 29..."
true,"{null, Mon Aug 29..."
true,"{null, Wed Aug 24..."
true,"{null, Sun Aug 28..."
true,"{null, Mon Aug 29..."
true,"{null, Sun Aug 28..."
true,"{null, Sun Aug 28..."
false,null
false,null
true,"{null, Mon Aug 29..."


In [51]:
tweets_df.filter((f.col('retweeted_status').isNull()) & (f.col('is_retweet').isNotNull())).select('is_retweet', 'retweeted_status').limit(10)

is_retweet,retweeted_status
false,null
false,null
false,null
false,null
false,null
false,null
false,null
false,null
false,null
false,null


In [52]:
# Original and retweets can both have null user_location
tweets_df.select('user_location', 'is_retweet').limit(10)

user_location,is_retweet
In kyuhyun's heart,true
"Norwich, England",true
"Atlanta, GA",true
"Lake City, FL",false
"New York, New York",false
Ehlalankosi nase ...,true
"Oneida, KY",false
null,true
null,false
"Pretoria, South A...",true


In [36]:
tweets_df.columns

['user_description',
 'user_favourites_count',
 'user_followers_count',
 'user_friends_count',
 'user_id',
 'user_name',
 'user_protected',
 'user_screen_name',
 'user_statuses_count',
 'user_verified',
 'user_withheld_in_countries',
 'user_location',
 'tweet_date',
 'retweeted_status',
 'is_retweet',
 'is_reply',
 'tweet_id',
 'retweeted_from',
 'tweet_text',
 'quoted_status',
 'hashtags',
 'tweet_coordinates',
 'retweet_count',
 'retweeted_from_name',
 'retweeted_from_description']

### Cleaning the Tweet Text

In [16]:
import nltk
nltk.download('omw-1.4')
nltk.download('stopwords')
nltk.download('wordnet')
from nltk.corpus import stopwords
from nltk.corpus import wordnet

[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [17]:
import re

In [18]:
stopwords_eng = stopwords.words('english')


@f.udf(returnType=StringType())
def clean_tweet(tweet):
    text = tweet.strip().lower()
    # Contractions like can't, won't, but also removes it's, mike's
    text = re.sub("'", "", text) 
    # Links
    text = re.sub(r'http\S+', '', text)
    text = re.sub(r"www.\S+", '', text)
    # Punctuation
    text = re.sub('[()!?]', ' ', text)
    text = re.sub('\[.*?\]',' ', text)
    # Only alphanumeric
    text = re.sub("[^a-z0-9]"," ", text)
    text = text.split()
    text = [word for word in text if not word in stopwords_eng]
    text = " ".join(word for word in text)
    return text

In [44]:
tweets_df.select('tweet_text').limit(10)

tweet_text
i tell college at...
in a conversation...
children of polit...
rwanda has increa...
@dpradhanbjp sir ...
these two are lik...
@starsootball gir...
two hindu childre...
new research from...
1/2 pls share! th...


In [19]:
tweets_df = tweets_df.withColumn('tweet_text', clean_tweet('tweet_text'))

In [46]:
tweets_df.select('tweet_text').limit(10)

tweet_text
tell college athl...
conversation toro...
children politici...
rwanda increased ...
dpradhanbjp sir r...
two like school k...
starsootball girl...
two hindu childre...
new research uark...
1 2 pls share uni...


In [20]:
@f.udf(returnType=StringType())
def clean_text(text):
    if text is None:
        return None
    text = text.strip().lower()
    # Contractions like can't, won't, but also removes it's, mike's
    text = re.sub("'", "", text) 
    # Links
    text = re.sub(r'http\S+', '', text)
    text = re.sub(r"www.\S+", '', text)
    # Only alphanumeric
    text = re.sub("[^a-z0-9]"," ", text)
    return text

In [21]:
tweets_df = (tweets_df.withColumn('retweeted_from_description', clean_text('retweeted_from_description'))
        .withColumn('user_description', clean_text('user_description')))

In [50]:
tweets_df.select(['user_description', 'retweeted_from_description']).limit(10)

user_description,retweeted_from_description
psu pittsburg...,host brenemansh...
toronto metropoli...,a dynamic faculty...
crypto music blo...,get to know the t...
just being me pr...,africas biggest f...
null,null
pharmacist and fa...,null
if yall see btslo...,null
a nationalist mi...,political analyst...
tufts 2017 clin...,twitter home for ...
chris paton iris...,postgraduate cert...


### Writing the Processed data to the disk

In [22]:
tweets_df.write.format("parquet").\
mode('overwrite').\
save('gs://' + 'msca-bdp-students-bucket/shared_data/pranavr569' + '/tweets_processed')

In [55]:
!hadoop fs -ls 'gs://msca-bdp-students-bucket/shared_data/pranavr569/tweets_processed' | head

Found 348 items
-rwx------   3 root root          0 2023-03-03 21:30 gs://msca-bdp-students-bucket/shared_data/pranavr569/tweets_processed/_SUCCESS
-rwx------   3 root root   41089665 2023-03-03 21:15 gs://msca-bdp-students-bucket/shared_data/pranavr569/tweets_processed/part-00000-04294563-ef36-402d-8400-054e1399dcb0-c000.snappy.parquet
-rwx------   3 root root   40402135 2023-03-03 21:15 gs://msca-bdp-students-bucket/shared_data/pranavr569/tweets_processed/part-00001-04294563-ef36-402d-8400-054e1399dcb0-c000.snappy.parquet
-rwx------   3 root root   39837356 2023-03-03 21:15 gs://msca-bdp-students-bucket/shared_data/pranavr569/tweets_processed/part-00002-04294563-ef36-402d-8400-054e1399dcb0-c000.snappy.parquet
-rwx------   3 root root   39866916 2023-03-03 21:15 gs://msca-bdp-students-bucket/shared_data/pranavr569/tweets_processed/part-00003-04294563-ef36-402d-8400-054e1399dcb0-c000.snappy.parquet
-rwx------   3 root root   39212877 2023-03-03 21:15 gs://msca-bdp-students-bucket/share